<a href="https://colab.research.google.com/github/andradeM17/Data-quality/blob/main/LLMs-as-judges/Round%202/qwen/run-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

**Qwen2.5-14B-Instruct**

## Load the model and data

---



We load the model from here: [qwen/qwen2.5-14B-Instruct](https://hf.co/qwen/qwen2.5-14B-Instruct)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "Qwen/Qwen2.5-14B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="cuda",
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd

llm_data = pd.read_csv('/content/drive/MyDrive/llmj-data.csv')
print("llmdata.csv loaded successfully.")


## Specify Reasoning Effort

In [ ]:
guidelines= '''You will be given a text, containing two parts
IF the first sample is "nan", it is a monolingual sample; ELSE the sample is parallel data.
Using the guidelines, give the correct code for the text: NL, WL, X, CS, CB, or CC. Finish your statement in the format:
'\n
Annotation: [value]
\n\n';


Guidelines:
  IF the content is NOT linguistic OR you are unsure THEN annotate as NL NOTE: ""No or unsure"" → NL
  ELSE IF the text is parallel data AND the target side is NOT in Modern (standardised) Irish OR the source side is NOT in Modern English THEN annotate as WL NOTE: ""No or Unsure"" → WL NOTE: Untranslated named entities on either side are treated as belonging to that language (e.g., ""Capnat"" on the English side counts as English) NOTE: If text is entirely named entities, it is more likely to be ‘Yes’
  ELSE IF the Irish target text is NOT a direct translation of the English source text THEN annotate as X NOTE: ""Yes or unsure"" → next question; if not, annotate X
  ELSE IF the Irish text is short (just headings, single unrelated phrases, OR five words or fewer) THEN annotate as CS NOTE: Short text = CS
  ELSE IF the text is boilerplate OR low quality (text that would repeat across similar webpages, unnatural code-switching, unnatural phrasing, frequent unnatural formatting, misalignments) THEN annotate as CB NOTE: CB includes text not representative of natural language One formatting problem alone is NOT enough; more than one may qualify
  ELSE annotate as CC NOTE: CC = natural language, not short, not boilerplate, properly aligned.'''




for index, row in llm_data.iterrows():
    text = f'"{str(row.iloc[0])}", "{str(row.iloc[1])}"'
    print(f"\n{text}\n\n")

    messages = [
        {"role": "system", "content": guidelines},
        {"role": "user", "content": text},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
    ).to(model.device)

    generated = model.generate(**inputs, max_new_tokens=500)
    decoded_output = tokenizer.decode(
        generated[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    print(decoded_output.strip())
